In [23]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict


In [24]:
class BatsmanState(TypedDict):
    runs: int
    balls:int
    fours:int
    sixes:int

    sr:float
    bpb:float
    boundary_percent: float
    summary : str

In [35]:
def calculate_sr(state : BatsmanState) -> BatsmanState:
    runs = state['runs']
    balls = state['balls']

    sr = (runs/balls)*100

    return {'sr':sr}

In [ ]:
def calculate_bpb(state : BatsmanState) -> BatsmanState:
    fours = state['fours']
    sixes = state['sixes']
    boundary = fours + sixes 
    balls = state['balls']

    bpb = (balls/boundary)

    return {'bpb':bpb}

In [36]:
def calculate_boundary_percent(state : BatsmanState) -> BatsmanState:
    fours = state['fours']
    sixes = state['sixes']
    runs_via_boundaries = fours*4 + sixes*6
    runs = state['runs']
    boundary_percent = (runs_via_boundaries/runs)*100
    return {'boundary_percent':boundary_percent}

In [ ]:
def summary(state:BatsmanState) -> BatsmanState:
    summary = f"""
Strike Rate = {state['sr']} \n
Balls = {state['balls']} \n
Runs = {state['runs']} \n
Fours = {state['fours']} \n
Sixes = {state['sixes']} \n
Boundary Percent = {state['boundary_percent']} \n
Balls per Boundary = {state['bpb']} \n
"""
    
    return {'summary':summary}

In [33]:
graph = StateGraph(BatsmanState)

graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percent', calculate_boundary_percent)
graph.add_node('summary', summary)

# edges
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundary_percent','summary')

graph.add_edge('summary',END)

workflow = graph.compile()

In [34]:
initial_state = {'runs':100,
                 'balls': 50,
                 'fours':8, 
                 'sixes':7}
 
final_state = workflow.invoke(initial_state)

InvalidUpdateError: Expected dict, got 3.33
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE